<a href="https://colab.research.google.com/github/eoproject2022/Palm-Oil/blob/main/Replicate_Tutorial11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 198 kB 52.9 MB/s 
     |████████████████████████████████| 99 kB 8.8 MB/s 
     |████████████████████████████████| 3.4 MB 35.8 MB/s 
     |████████████████████████████████| 130 kB 38.4 MB/s 
     |████████████████████████████████| 1.2 MB 63.5 MB/s 
     |████████████████████████████████| 3.0 MB 31.5 MB/s 
     |████████████████████████████████| 98 kB 6.5 MB/s 
     |████████████████████████████████| 8.8 MB 33.1 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 1.3 MB 48.0 MB/s 
     |████████████████████████████████| 128 kB 44.8 MB/s 
     |████████████████████████████████| 1.6 MB 43.1 MB/s 
     |████████████████████████████████| 423 kB 42.3 MB/s 
     |████████████████████████████████| 

In [1]:
import ee
import geemap
import os

In [2]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=aR5lzdQTUyftoqC5mft0r0wpVYHi6Ehj7_ouoLDSoWY&tc=Kqh6adrZczBt4hxpIjyQ_saXjmrFY0Rz6BljO3h-xdQ&cc=cJHnaiJfs8vQlu9fzSbWV5275ox9tFFatXDeDZJsw8M

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJoZLjZ-qF8rIjbEEQ9xSd7XYrZ462KwhjkYXPfVdBCS8BqRD-jLAQ8

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Download an ee.Image

In [3]:
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(image, landsat_vis, "LE7_TOA_5YEAR/1999_2003", True, 0.7)

In [4]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature

if feature is None:
    geom = ee.Geometry.Polygon(
        [
            [
                
              [102.79095636432051,2.4894862771783317],
              [102.82803522174238,2.4893147778027087],
              [102.82820688311934,2.521213276063402],
              [102.80503259723066,2.5291020317543977],
              [102.79129968707441,2.520355799734292],
              [102.79095636432051,2.4894862771783317],
            ]
        ]
    )
    feature = ee.Feature(geom, {})

roi = feature.geometry()

In [5]:
out_dir = os.path.join(os.path.expanduser('c:/mockdownload'), 'Downloads')
filename = os.path.join(out_dir, 'landsat.tif')

### Exporting all bands as one single image

In [6]:
image = image.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=90, region=roi, file_per_band=False
)

Generating URL ...
Please wait ...
An error occurred while downloading.


JSONDecodeError: ignored

In [9]:
import requests
response = requests.get("https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/b7808083910a70c0e0cd4693a9035e22-ed4c9f138db441611796f5ba7918913f:getPixels")
response.raise_for_status()
response.json()

JSONDecodeError: ignored

### Exporting each band as one image

In [7]:
geemap.ee_export_image(
    image, filename=filename, scale=90, region=roi, file_per_band=True
)

Generating URL ...
Please wait ...
An error occurred while downloading.


JSONDecodeError: ignored

### Export an image to Google Drive

In [ ]:
geemap.ee_export_image_to_drive(
    image, description='landsat', folder='ILSM', region=roi, scale=30
)

## Download an ee.ImageCollection

In [ ]:
import ee
import geemap
import os

In [ ]:
loc = ee.Geometry.Point(102.82820688311934,2.521213276063402)
collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(loc)
    .filterDate('2008-01-01', '2020-01-01')
    .filter(ee.Filter.listContains("system:band_names", "N"))
)

In [ ]:
out_dir = os.path.join(os.path.expanduser('c:/mockdownload'), 'Downloads')

In [ ]:
print(collection.aggregate_array('system:index').getInfo())

[]


In [ ]:
geemap.ee_export_image_collection(collection, out_dir=out_dir)

Total number of images: 0



In [ ]:
geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=10)

Total number of images: 0



## Extract pixels as a Numpy array

In [ ]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810').select(['B4', 'B5', 'B6'])

aoi = ee.Geometry.Polygon(
    [[  [102.81030610944818,2.5260462435775226],
        [102.81030610944818,2.5274181994267853],
        [102.8078599348266,2.5274181994267853],
        [102.8078599348266,2.5260462435775226]]], None, False
)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'B6' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().


AttributeError: ignored

In [ ]:
# Scale the data to [0, 255] to show as an RGB image.
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255 * ((rgb_img[:, :, 0:3] - 100) / 3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()